## Imports 

In [21]:
import argparse
import easydict
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from matplotlib import pyplot as plt
from plotread import *
import shap
import pickle
from collections import Counter
%matplotlib inline
%config NotebookApp.iopub_data_rate_limit = 2000000.0
%config NotebookApp.rate_limit_window = 5.0


# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# init the JS visualization code
shap.initjs()

In [3]:
import tensorflow._api.v2.compat.v1 as tf                 # does not work for python=3.7
from tensorflow.compat.v1.keras.backend import get_session
tf.compat.v1.disable_v2_behavior()                          # does not work for thesis, except with DeepExplainer
tf.compat.v1.disable_eager_execution()                    # does not work

from tensorflow.python.ops.numpy_ops import np_config     # does not work for python=3.7
np_config.enable_numpy_behavior()                         # does not work for python=3.7

Instructions for updating:
non-resource variables are not supported in the long term


## Functions 

In [4]:
def main():
    ###########################################################################
    # Parser Definition
    ###########################################################################
    opt = easydict.EasyDict({
        "model": "GRU",
        "datapath": "Dataset1/",
        "savepath": "Experiment1/GRU/8/Singleoutput/Runy/",
        "extension": ".dat",
        "batch_size": 32,
        "plots_in": False,
        "plots_out": True
    })

    ###########################################################################
    # Variables Definition
    ###########################################################################
    nin = ['time', 'PLML2', 'PLMR', 'AVBL', 'AVBR']
    nout = ['time', 'DB1', 'LUAL', 'PVR', 'VB1']
    neurons = ['time','DB1','LUAL','PVR','VB1','PLML2','PLMR','AVBL','AVBR']

    ###########################################################################
    # Read data
    ###########################################################################
    files = getdata(opt.datapath, opt.extension)
    train, valid, test = splitdata(files)
    trainx, trainy = readdata(opt.datapath, train, neurons, nin, nout)
    validx, validy = readdata(opt.datapath, valid, neurons, nin, nout)
    testx, testy = readdata(opt.datapath, test, neurons, nin, nout)
    if opt.plots_in:
        plotdata(trainx, '/train_data', '/x', opt.model, opt.savepath)
        plotdata(trainy, '/train_data', '/y', opt.model, opt.savepath)
        plotdata(validx, '/valid_data', '/x', opt.model, opt.savepath)
        plotdata(validy, '/valid_data', '/y', opt.model, opt.savepath)
        plotdata(testx, '/test_data', '/x', opt.model, opt.savepath)
        plotdata(testy, '/test_data', '/y', opt.model, opt.savepath)
        
    ###########################################################################
    # Load Model and Evaluate
    ###########################################################################
    output_size = 1
    global model
    model = load_model(opt.savepath + 'model.h5')
    model.summary()

    print("Starting to explain...")
    X_train = np.array(trainx)
    X_test = np.array(testx)

    nin_names = nin[1:len(nin)]

    return X_train, X_test, nin_names

## Code 

In [5]:
X_train, X_test, nin_names = main()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 1000, 8)           336       
                                                                 
 time_distributed (TimeDistr  (None, 1000, 1)          9         
 ibuted)                                                         
                                                                 
 flatten (Flatten)           (None, 1000)              0         
                                                                 
Total params: 345
Trainable params: 345
Non-trainable params: 0
_________________________________________________________________
Starting to explain...


In [6]:
X_train.shape

(20, 1000, 4)

In [7]:
X_test.shape

(10, 1000, 4)

In [8]:
background = X_train[np.random.choice(X_train.shape[0], 1, replace=False)]
print("background.shape=")
print(background.shape)

background.shape=
(1, 1000, 4)


In [9]:
background

array([[[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]])

In [10]:
for layer in model.layers:
    print("LAYER")
    print(layer.name, layer.inbound_nodes, layer.outbound_nodes)
    print(layer.input_shape)
    print(layer.output_shape) 

LAYER
gru [<keras.engine.node.Node object at 0x00000199F9F44F70>] [<keras.engine.node.Node object at 0x00000199F9F94370>]
(None, 1000, 4)
(None, 1000, 8)
LAYER
time_distributed [<keras.engine.node.Node object at 0x00000199F9F94370>] [<keras.engine.node.Node object at 0x00000199F9FF8C70>]
(None, 1000, 8)
(None, 1000, 1)
LAYER
flatten [<keras.engine.node.Node object at 0x00000199F9FF8C70>] []
(None, 1000, 1)
(None, 1000)


In [11]:
X_test[:1,:,:]

array([[[1.4, 0. , 2.3, 0. ],
        [1.4, 0. , 2.3, 0. ],
        [1.4, 0. , 2.3, 0. ],
        ...,
        [0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. ]]])

### explainers

In [12]:
# explainer1 = shap.DeepExplainer(model, X_train)
# explainer2 = shap.DeepExplainer(model, background)

explainer3 = shap.DeepExplainer((model.layers[0].input, model.layers[-1].output), X_train)
explainer4 = shap.DeepExplainer((model.layers[0].input, model.layers[-1].output), background)

# explainer5 = shap.GradientExplainer(model, X_train)
# explainer6 = shap.GradientExplainer(model, background)

# explainer7 = shap.GradientExplainer((model.layers[0].input, model.layers[-1].output), X_train)
# explainer8 = shap.GradientExplainer((model.layers[0].input, model.layers[-1].output), background)

### shap_values

#### Loading the shap_values from pickle files

In [ ]:
# with open("D:\Pedro\IST\ANO 2\SEM 2\Tese\gmestre\ICLR-RNN-CElegans\Experiment1\GRU\8\Singleoutput\Runy\pickles\shap_values1_explainer4.pkl", 'rb') as file:
#     shap_values1_explainer4 = pickle.load(file)

In [ ]:
# list_of_shap_values = []

# for i in range(1, 9):
#     for j in range(1, 3):
#         with open("D:\Pedro\IST\ANO 2\SEM 2\Tese\gmestre\ICLR-RNN-CElegans\Experiment1\GRU\8\Singleoutput\Runy\pickles\shap_values{}_explainer{}.pkl".format(j, i), 'rb') as file:
#             shap_value = pickle.load(file)
#             list_of_shap_values.append(shap_value)

In [ ]:
# shap_values1_explainer1 = list_of_shap_values[0]
# shap_values2_explainer1 = list_of_shap_values[1]
# shap_values1_explainer2 = list_of_shap_values[2]
# shap_values2_explainer2 = list_of_shap_values[3]
# shap_values1_explainer3 = list_of_shap_values[4]
# shap_values2_explainer3 = list_of_shap_values[5]
# shap_values1_explainer4 = list_of_shap_values[6]
# shap_values2_explainer4 = list_of_shap_values[7]
# shap_values1_explainer5 = list_of_shap_values[8]
# shap_values2_explainer5 = list_of_shap_values[9]
# shap_values1_explainer6 = list_of_shap_values[10]
# shap_values2_explainer6 = list_of_shap_values[11]
# shap_values1_explainer7 = list_of_shap_values[12]
# shap_values2_explainer7 = list_of_shap_values[13]
# shap_values1_explainer8 = list_of_shap_values[14]
# shap_values2_explainer8 = list_of_shap_values[15]

In [13]:
list_of_shap_values = []

for i in range(3, 5):
    for j in range(1, 3):
        with open("D:\Pedro\IST\ANO 2\SEM 2\Tese\gmestre\ICLR-RNN-CElegans\Experiment1\GRU\8\Singleoutput\Runy\pickles\shap_values{}_explainer{}.pkl".format(j, i), 'rb') as file:
            shap_value = pickle.load(file)
            list_of_shap_values.append(shap_value)

In [14]:
shap_values1_explainer3 = list_of_shap_values[0]
shap_values2_explainer3 = list_of_shap_values[1]
shap_values1_explainer4 = list_of_shap_values[2]
shap_values2_explainer4 = list_of_shap_values[3]

#### Calculating shap_values
Below we have used explainer to generate shape values for the test dataset using the shap_values() method of explainer. The explainer object has a base value to which it adds shape values for a particular sample in order to generate a final prediction. The base value is stored in the expected_value attribute of the explainer object. All model predictions will be generated by adding shap values generated for a particular sample to this expected value.

In [ ]:
# shap_values1_explainer1 = explainer1.shap_values(X_test[:1,:,:])

In [ ]:
# shap_values2_explainer1 = explainer1.shap_values(X_test)

In [ ]:
# shap_values1_explainer2 = explainer2.shap_values(X_test[:1,:,:])

In [ ]:
# shap_values2_explainer2 = explainer2.shap_values(X_test)

In [ ]:
# shap_values1_explainer3 = explainer3.shap_values(X_test[:1,:,:])

In [ ]:
# shap_values2_explainer3 = explainer3.shap_values(X_test)

In [ ]:
# shap_values1_explainer4 = explainer4.shap_values(X_test[:1,:,:])

In [ ]:
# shap_values2_explainer4 = explainer4.shap_values(X_test)

In [ ]:
# shap_values1_explainer5 = explainer5.shap_values(X_test[:1,:,:])

In [ ]:
# shap_values2_explainer5 = explainer5.shap_values(X_test)

In [ ]:
# shap_values1_explainer6 = explainer6.shap_values(X_test[:1,:,:])

In [ ]:
# shap_values2_explainer6 = explainer6.shap_values(X_test)

In [ ]:
# shap_values1_explainer7 = explainer7.shap_values(X_test[:1,:,:])

In [ ]:
# shap_values2_explainer7 = explainer7.shap_values(X_test)

In [ ]:
# shap_values1_explainer8 = explainer8.shap_values(X_test[:1,:,:])

In [ ]:
# shap_values2_explainer8 = explainer8.shap_values(X_test)

#### Creating pickle files to save the shap_values

In [ ]:
# with open("D:\Pedro\IST\ANO 2\SEM 2\Tese\gmestre\ICLR-RNN-CElegans\Experiment1\GRU\8\Singleoutput\Runy\pickles\shap_values1_explainer4.pkl", 'wb') as file:
#     pickle.dump(shap_values1_explainer4, file)

In [ ]:
# for i in range(3, 5):
#     for j in range(1, 3):
#         with open("D:\Pedro\IST\ANO 2\SEM 2\Tese\gmestre\ICLR-RNN-CElegans\Experiment1\GRU\8\Singleoutput\Runy\pickles\shap_values{}_explainer{}.pkl".format(j, i), 'wb') as file:
#             pickle.dump(eval("shap_values{}_explainer{}".format(j, i)), file)

In [ ]:
for i in range(1, 9):
    for j in range(1, 3):
        with open("D:\Pedro\IST\ANO 2\SEM 2\Tese\gmestre\ICLR-RNN-CElegans\Experiment1\GRU\8\Singleoutput\Runy\pickles\shap_values{}_explainer{}.pkl".format(j, i), 'wb') as file:
            pickle.dump(eval("shap_values{}_explainer{}".format(j, i)), file)

#### Print shap_values

In [ ]:
# print(shap_values1_explainer1)
# print(shap_values2_explainer1)

In [ ]:
# print(shap_values1_explainer2)
# print(shap_values2_explainer2)

In [56]:
for i in shap_values2_explainer3[0]
print(np.array_equal(shap_values1_explainer3[0][0], i))

<generator object <genexpr> at 0x00000199A7230430>


In [53]:
res = any(np.array_equal(shap_values1_explainer3[0][0], i) for i in shap_values2_explainer3[0])

In [54]:
res

True

In [15]:
print(shap_values1_explainer3)

[array([[[ 0.0235888 , -0.00426877, -0.00149598, -0.00096539],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ]]]), array([[[ 0.01785798, -0.00347505,  0.00202627, -0.00115146],
        [ 0.0201146 , -0.0031999 , -0.00259696, -0.00059117],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ]]]), array([[[ 0.01360732, -0.00288643,  0.00400891, -0.00126968],
        [ 0.01505574, -0.00263172,  0.00126887, -0.00085239],
        [ 0.01727988, -0.00235855, -0.00339144, -0.00035299],
        ...,
        [ 0.        ,  0

In [22]:
print(shap_values2_explainer3)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [17]:
print(shap_values1_explainer4)

[array([[[ 3.20101254e-02, -2.06345912e-06,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00]]]), array([[[ 2.60528561e-02, -1.54183999e-06,  0.00000000e+00,
          0.00000000e+00],
        [ 2.59148642e-02, -6.51420591e-05,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.0

In [20]:
print(shap_values2_explainer4)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# print(shap_values1_explainer5)
# print(shap_values2_explainer5)

In [ ]:
# print(shap_values1_explainer6)
# print(shap_values2_explainer6)

In [ ]:
# print(shap_values1_explainer7)
# print(shap_values2_explainer7)

In [ ]:
# print(shap_values1_explainer8)
# print(shap_values2_explainer8)

In [48]:
# print(np.array(shap_values1_explainer1).shape)
# print(np.array(shap_values2_explainer1).shape)

# print(np.array(shap_values1_explainer2).shape)
# print(np.array(shap_values2_explainer2).shape)

print(np.array(shap_values1_explainer3).shape)
print(np.array(shap_values2_explainer3).shape)

print(np.array(shap_values1_explainer4).shape)
print(np.array(shap_values2_explainer4).shape)

# print(np.array(shap_values1_explainer5).shape)
# print(np.array(shap_values2_explainer5).shape)

# print(np.array(shap_values1_explainer6).shape)
# print(np.array(shap_values2_explainer6).shape)

# print(np.array(shap_values1_explainer7).shape)
# print(np.array(shap_values2_explainer7).shape)

# print(np.array(shap_values1_explainer8).shape)
# print(np.array(shap_values2_explainer8).shape)

(1000, 1, 1000, 4)
(1000, 10, 1000, 4)
(1000, 1, 1000, 4)
(1000, 10, 1000, 4)


In [ ]:
shap_values1_explainer3_reshaped = np.reshape(np.array(shap_values1_explainer3), (1000, 1000, 4))

In [ ]:
shap_values1_explainer3_reshaped.shape

In [ ]:
shap_values1_explainer4_reshaped = np.reshape(np.array(shap_values1_explainer4), (1000, 1000, 4))

In [ ]:
shap_values1_explainer4_reshaped.shape

### expected_values

In [ ]:
# print(explainer1.expected_value)
# print(explainer2.expected_value)
print(explainer3.expected_value)
print(explainer4.expected_value)

In [ ]:
# print(explainer1.expected_value.shape)
# print(explainer2.expected_value.shape)
print(explainer3.expected_value.shape)
print(explainer4.expected_value.shape)

## Visualizations

### dependence_plots

##### Code for X_test[:1,:,:]

In [ ]:
# shap.dependence_plot(
#             ind=0,
#             shap_values=shap_values1_explainer3_reshaped[0],
#             features=X_test[:1,:,:][0],
#             feature_names=nin_names,
#             interaction_index=1)

In [ ]:
# shap.dependence_plot(
#             ind=1,
#             shap_values=shap_values1_explainer3_reshaped[0],
#             features=X_test[:1,:,:][0],
#             feature_names=nin_names,
#             interaction_index=3)

In [ ]:
# for i in range(shap_values1_explainer3_reshaped.shape[0]):
#     for j in range(len(nin_names)):
#         print(i)
#         print(j)
#         shap.dependence_plot(
#                 ind=j,
#                 shap_values=shap_values1_explainer3_reshaped[i],
#                 features=X_test[0],
#                 feature_names=nin_names)

##### Code for X_test

In [ ]:
# for k in  range(X_test.shape[0]):
#     for i in range(np.array(shap_values2_explainer3).shape[0]):
#         for j in range(len(nin_names)):
#             print(k)
#             print(i)
#             print(j)
#             shap.dependence_plot(
#                     ind=j,
#                     shap_values=shap_values2_explainer3[i][k],
#                     features=X_test[k],
#                     feature_names=nin_names)

### bar_plots
It shows a bar plot of shap values' impact on the prediction of a particular sample.

##### Code for X_test[:1,:,:]

In [ ]:
# for i in range(len(shap_values1_explainer3)):
#     print(i)
#     shap.bar_plot(
#         shap_values=shap_values1_explainer3_reshaped[i][0],
#         features=X_test[:1,:,:][0][0],
#         feature_names=nin_names)

In [ ]:
# for i in range(len(shap_values1_explainer3_reshaped)):
#     print(i)
#     shap.bar_plot(
#         shap_values=shap_values1_explainer3_reshaped[i][i],
#         features=X_test[:1,:,:][0][0],
#         feature_names=nin_names)

##### Code for X_test

In [ ]:
# for i in range(len(shap_values2_explainer3)):
#     for j in range(len(shap_values2_explainer3[0])):
#         print(i)
#         print(j)
#         shap.bar_plot(
#             shap_values=shap_values2_explainer3[i][j][0],
#             features=X_test[j][0],
#             feature_names=nin_names)

### plots.force

##### Code for X_test[:1,:,:]

In [ ]:
# shap.plots.force(
#     base_value=explainer3.expected_value[0],
#     shap_values=shap_values1_explainer3_reshaped[0],
#     features=X_test[:1,:,:][0],
#     feature_names=nin_names)

In [ ]:
# shap.plots.force(
#     base_value=explainer3.expected_value[500],
#     shap_values=shap_values1_explainer3_reshaped[500],
#     features=X_test[:1,:,:][0],
#     feature_names=nin_names)

In [ ]:
# shap.plots.force(
#     base_value=explainer3.expected_value[999],
#     shap_values=shap_values1_explainer3_reshaped[999],
#     features=X_test[:1,:,:][0],
#     feature_names=nin_names)

##### Code for X_test

In [ ]:
# shap.plots.force(
#     base_value=explainer3.expected_value[0],
#     shap_values=shap_values2_explainer3[0][9],
#     features=X_test[9],
#     feature_names=nin_names)

In [ ]:
# shap.plots.force(
#     base_value=explainer3.expected_value[500],
#     shap_values=shap_values2_explainer3[500][9],
#     features=X_test[9],
#     feature_names=nin_names)

In [ ]:
# shap.plots.force(
#     base_value=explainer3.expected_value[999],
#     shap_values=shap_values2_explainer3[999][9],
#     features=X_test[9],
#     feature_names=nin_names)

## force_plots
It plots shap values using additive force layout. It can help us see which features most positively or negatively contributed to prediction.

##### Code for X_test[:1,:,:]

In [ ]:
# shap.force_plot(
#     explainer3.expected_value[0],
#     shap_values1_explainer3[0][0],
#     nin_names)

In [ ]:
# shap.force_plot(
#     explainer3.expected_value[500],
#     shap_values1_explainer3[500][0],
#     nin_names)

In [ ]:
# shap.force_plot(
#     explainer3.expected_value[999],
#     shap_values1_explainer3[999][0],
#     nin_names)

##### Code for X_test

In [ ]:
# shap.force_plot(
#     explainer3.expected_value[0],
#     shap_values2_explainer3[0][9],
#     nin_names)

In [ ]:
# shap.force_plot(
#     explainer3.expected_value[500],
#     shap_values2_explainer3[500][9],
#     nin_names)

In [ ]:
# shap.force_plot(
#     explainer3.expected_value[999],
#     shap_values2_explainer3[999][9],
#     nin_names)

#### Ensure the sum of Shapley values matches the difference between predicted value and expected value: For each instance in your dataset, the sum of the Shapley values across all features should add up to the difference between the model's predicted value for that instance and the expected value.

In [ ]:
np.array(shap_values1_explainer3).shape

In [ ]:
np.array(shap_values2_explainer3).shape

In [ ]:
explainer3.expected_value.shape

In [ ]:
np.array(shap_values1_explainer4).shape

In [ ]:
np.array(shap_values2_explainer4).shape

In [ ]:
explainer4.expected_value.shape

In [ ]:
predicted_values_1 = model.predict(X_test[:1,:,:])
predicted_values_1.shape

In [ ]:
predicted_values_10 = model.predict(X_test)
predicted_values_10.shape

In [ ]:
for i in range(len(X_test[:1,:,:])):
    for j in range(predicted_values_1.shape[1]):
        shap_sum = np.sum(shap_values1_explainer3[j][i])
        pred_diff = predicted_values_1[i, j] - explainer3.expected_value[j]
        print(f"Instance {i+1}, Output {j+1}: Shapley sum = \t{shap_sum}, \n\tPredicted difference = \t\t{pred_diff}")

In [ ]:
for i in range(len(X_test)):
    for j in range(predicted_values_10.shape[1]):
        shap_sum = np.sum(shap_values2_explainer3[j][i])
        pred_diff = predicted_values_10[i, j] - explainer3.expected_value[j]
        print(f"Instance {i+1}, Output {j+1}: Shapley sum = \t{shap_sum}, \n\tPredicted difference = \t\t{pred_diff}")

In [ ]:
for i in range(len(X_test[:1,:,:])):
    for j in range(predicted_values_1.shape[1]):
        shap_sum = np.sum(shap_values1_explainer4[j][i])
        pred_diff = predicted_values_1[i, j] - explainer4.expected_value[j]
        print(f"Instance {i+1}, Output {j+1}: Shapley sum = \t{shap_sum}, \n\tPredicted difference = \t\t{pred_diff}")

In [ ]:
for i in range(len(X_test)):
    for j in range(predicted_values_10.shape[1]):
        shap_sum = np.sum(shap_values2_explainer4[j][i])
        pred_diff = predicted_values_10[i, j] - explainer4.expected_value[j]
        print(f"Instance {i+1}, Output {j+1}: Shapley sum = \t{shap_sum}, \n\tPredicted difference = \t\t{pred_diff}")

In [ ]:
def verify_shap_values(shap_values, expected_values, predicted_values):
    """
    Verify the correctness of SHAP values.

    Args:
        shap_values (numpy.ndarray): Array of SHAP values with shape (n_instances, 1, n_features, n_classes).
        expected_values (numpy.ndarray): Array of expected values with shape (n_instances,).
        predicted_values (numpy.ndarray): Array of predicted values with shape (1, n_instances).

    Returns:
        bool: True if the SHAP values are correctly calculated, False otherwise.
    """
    
    # Step 1: Verify shapes
    n_instances, _, n_features, n_classes = shap_values.shape
    _, n_instances_predicted = predicted_values.shape

    if n_instances != n_instances_predicted:
        print("Error: Number of instances in shap_values and predicted_values do not match.")
        return False

    # Step 2: Check the sum property
    for i in range(n_instances):
        sum_shap_values = np.sum(shap_values[i, 0, :, :])
        diff_predicted_expected = predicted_values[0, i] - expected_values[i]
        
        print(sum_shap_values)
        print(diff_predicted_expected)

        if not np.isclose(sum_shap_values, diff_predicted_expected, rtol=1e-3, atol=1e-5):
            print(f"Error: Sum of SHAP values for instance {i} does not match the difference between predicted and expected values.")
            return False

    return True

In [ ]:
verify_shap_values(np.array(shap_values1_explainer3), explainer3.expected_value, predicted_values_1)

In [ ]:
verify_shap_values(np.array(shap_values2_explainer3), explainer3.expected_value, predicted_values_10)

In [ ]:
verify_shap_values(np.array(shap_values1_explainer4), explainer4.expected_value, predicted_values_1)

In [ ]:
verify_shap_values(np.array(shap_values2_explainer4), explainer4.expected_value, predicted_values_10)

In [ ]:
def verify_shap_values2(shap_values, expected_values, predicted_values, threshold=1e-4):
    """
    Verify the correctness of SHAP values.

    Args:
        shap_values (numpy.ndarray): Array of SHAP values with shape (n_instances, 1, n_features, n_classes).
        expected_values (numpy.ndarray): Array of expected values with shape (n_instances,).
        predicted_values (numpy.ndarray): Array of predicted values with shape (1, n_instances).
        threshold (float): Threshold for comparing floating-point values.

    Returns:
        bool: True if the SHAP values are correctly calculated, False otherwise.
    """

    # Step 1: Verify shapes
    n_instances, _, n_features, n_classes = shap_values.shape
    _, n_instances_predicted = predicted_values.shape

    if n_instances != n_instances_predicted:
        print("Error: Number of instances in shap_values and predicted_values do not match.")
        return False

    # Step 2: Check the sum property for each instance
    for i in range(n_instances):
        sum_shap_values = np.sum(shap_values[i, 0, :, :])
        diff_predicted_expected = predicted_values[0, i] - expected_values[i]

        if not np.isclose(sum_shap_values, diff_predicted_expected, rtol=0, atol=threshold):
            print(f"Error: Sum of SHAP values for instance {i} does not match the difference between predicted and expected values.")
            return False

    # Step 3: Check consistency within each feature
    for i in range(n_features):
        sum_feature_values = np.sum(shap_values[:, 0, i, :])
        if not np.isclose(sum_feature_values, 0, rtol=0, atol=threshold):
            print(f"Error: Sum of SHAP values for feature {i} is not close to zero.")
            return False

    return True


In [ ]:
verify_shap_values2(np.array(shap_values1_explainer3), explainer3.expected_value, predicted_values_1)

In [ ]:
verify_shap_values2(np.array(shap_values2_explainer3), explainer3.expected_value, predicted_values_10)

In [ ]:
verify_shap_values2(np.array(shap_values1_explainer4), explainer4.expected_value, predicted_values_1)

In [ ]:
verify_shap_values2(np.array(shap_values2_explainer4), explainer4.expected_value, predicted_values_10)